# Creating an Synthetic NWB File

This tutorial is taken from the online example at http://pynwb.readthedocs.io/en/latest/example.html

In [1]:
import numpy as np

rate = 10.0
np.random.seed(1234)
data_len = 1000
ephys_data = np.random.rand(data_len)
ephys_timestamps = np.arange(data_len) / rate
spatial_timestamps = ephys_timestamps[::10]
spatial_data = np.cumsum(np.random.normal(size=(2, len(spatial_timestamps))), axis=-1).T

In [2]:
from datetime import datetime
from pynwb import NWBFile

f = NWBFile('the PyNWB tutorial', 'my first synthetic recording', 'EXAMPLE_ID', datetime.now(),
            experimenter='Dr. Bilbo Baggins',
            lab='Bag End Laboratory',
            institution='University of Middle Earth at the Shire',
            experiment_description='I went on an adventure with thirteen dwarves to reclaim vast treasures.',
            session_id='LONELYMTN')

In [3]:
epoch_tags = ('example_epoch',)

ep1 = f.create_epoch(source='an hypothetical source', name='epoch1', start=0.0, stop=1.0,
                     tags=epoch_tags,
                     description="the first test epoch")

ep2 = f.create_epoch(source='an hypothetical source', name='epoch2', start=0.0, stop=1.0,
                     tags=epoch_tags,
                     description="the second test epoch")

In [4]:
device = f.create_device(name='trodes_rig123', source="a source")

electrode_name = 'tetrode1'
source = "an hypothetical source"
description = "an example tetrode"
location = "somewhere in the hippocampus"

electrode_group = f.create_electrode_group(electrode_name,
                                           source=source,
                                           description=description,
                                           location=location,
                                           device=device)


for idx in [1, 2, 3, 4]:
    f.add_electrode(idx,
                    x=1.0, y=2.0, z=3.0,
                    imp=float(-idx),
                    location='CA1', filtering='none',
                    description='channel %s' % idx, group=electrode_group)

electrode_table_region = f.create_electrode_table_region([0, 2], 'the first and third electrodes')

In [5]:


from pynwb.ecephys import ElectricalSeries
from pynwb.behavior import SpatialSeries

ephys_ts = ElectricalSeries('test_ephys_data',
                            'an hypothetical source',
                            ephys_data,
                            electrode_table_region,
                            timestamps=ephys_timestamps,
                            # Alternatively, could specify starting_time and rate as follows
                            # starting_time=ephys_timestamps[0],
                            # rate=rate,
                            resolution=0.001,
                            comments="This data was randomly generated with numpy, using 1234 as the seed",
                            description="Random numbers generated with numpy.random.rand")
f.add_acquisition(ephys_ts, [ep1, ep2])

In [6]:
import pynwb

In [7]:
pynwb.__version__

u'0.2.0.post.dev416'

In [8]:
from pynwb import get_manager
from pynwb.form.backends.hdf5 import HDF5IO

filename = "example.h5"
io = HDF5IO(filename, manager=get_manager(), mode='w')
io.write(f)
io.close()

In [9]:
filename = "example.h5"
io = HDF5IO(filename, manager=get_manager(), mode='r')
f = io.read()


Exception: Could not construct Epoch object
    incorrect type for 'tags' (got 'H5Dataset', expected 'tuple or list'): Epoch (epoch2)